In [204]:
import pandas as pd
import numpy as np
from collections import deque
import copy
from tqdm import tqdm_notebook as tqdm
import time

In [9]:
# ナップサック問題を作る関数
def generate_problem(N, RAND_MAX):
    value = np.random.randint(1, RAND_MAX, N)
    weight = np.random.randint(1, RAND_MAX, N)
    ratio = value / weight
    capacity = N * RAND_MAX / 6
    return [value, weight, ratio, capacity]

In [126]:
def binary(i, X, res):
    if i >= N:
        if weight @ X <= capacity:
            return [value @ X, np.array(X)]
    else:
        X[i] = 0
        res += binary(i+1, X, [])
        X[i] = 1
        res += binary(i+1, X, [])
    return res

def enumeration():
    enum = np.array(binary(0, np.array([0 for i in range(N)]), [])).reshape(-1,2)
    enum_T = enum.T
    return enum[np.argsort(enum_T[0])[-1]]

In [10]:
def greedy(problem):
    value = problem[0]
    weight = problem[1]
    ratio = problem[2]
    capacity = problem[3]
    
    ratio_index = np.argsort(ratio)[::-1]
    w = 0
    v = 0
    i = 0
    ans = [0 for i in range(N)]
    while i < N:
        if w + weight[ratio_index[i]] <= capacity:
            w += weight[ratio_index[i]]
            ans[ratio_index[i]] = 1
            v += value[ratio_index[i]]
            i += 1
        else:
            i += 1
    return [v,ans]

In [237]:
def relaxation(problem, fix):
    value = problem[0]
    weight = problem[1]
    ratio = problem[2]
    capacity = problem[3]
    
    ratio_index = np.argsort(ratio)[::-1]
    w = sum([item[1]*weight[item[0]] for item in fix if item[1]==1])
    v = sum([item[1]*value[item[0]] for item in fix if item[1]==1])
    ans = [0 for i in range(len(value))]
    for item in fix:
        if item[1] == 1:
            ans[item[0]] = 1
    fix_index = [item[0] for item in fix]
    i = 0
    
    while i < N:
        if ratio_index[i] not in fix_index:
            if w + weight[ratio_index[i]] <= capacity:
                w += weight[ratio_index[i]]
                ans[ratio_index[i]] = 1
                v += value[ratio_index[i]]
                i += 1
            else:
                ans[ratio_index[i]] = (capacity - w) / weight[ratio_index[i]]
                v += ((capacity - w) / weight[ratio_index[i]]) * value[ratio_index[i]]
                break
        else:
            i += 1
    return [v, ans]

In [247]:
def prued_relax(problem, fix):  
    relax_cnt = 0
    
    value = problem[0]
    weight = problem[1]
    ratio = problem[2]
    capacity = problem[3]
    
    fix_index = [item[0] for item in fix]
    
    # 固定されてない変数を一個取ってくる
    for i in range(len(value)):
        if i not in fix_index:
            stack = deque([[i,0],[i,1]])
            break
   
    ans = deque([])
    # 暫定解
    temp_value = 0
    while stack:
        temp = stack.pop()
        i = temp[0]
        x_i = temp[1]

        ans.append([i, x_i])

        # 許容解がない
        if sum([w[1]*weight[w[0]] for w in fix+list(ans) if w[1]==1]) > capacity:
            while ans:
                flag = ans.pop()
                if flag[1] == 1:
                    break
        else:
            # 線形緩和を解く
            relax_cnt += 1
            ans_v,ans_w = relaxation(problem, fix+list(ans))
            not_Z_list = [i for i,w in enumerate(ans_w) if (w!=1 and w!=0)]
            # 整数解が見つかった 
            if sum(not_Z_list) == 0:
                # 暫定解より良い解ならば更新する
                if ans_v > temp_value:
                    temp_w = copy.copy(ans_w)
                    temp_value = ans_v
                while ans:
                    flag = ans.pop()
                    if flag[1] == 1:
                        break

            else:
                # 暫定解より大きければ深掘りする
                if ans_v > temp_value:
                    # 分数であるインデックスを取り出す
                    stack.append([not_Z_list[0],0])
                    stack.append([not_Z_list[0],1])
                else:
                    while ans:
                        flag = ans.pop()
                        if flag[1] == 1:
                            break           
    print(relax_cnt)
    return [temp_value, temp_w]

In [271]:
print("item数:")
N = int(input())
problem = generate_problem(N, 1000)
capacity = problem[3]

problem_df = pd.DataFrame(np.array(problem[:-1]).T, columns=["value","weight","ratio"])
problem_df = problem_df.sort_values(["ratio","weight"], ascending=[False,True])

print("capacity:",problem[-1])
print(problem_df)

item数:
100
capacity: 16666.666666666668
    value  weight      ratio
36  476.0    33.0  14.424242
15  710.0    56.0  12.678571
40  794.0    66.0  12.030303
30  904.0    76.0  11.894737
44  665.0    62.0  10.725806
..    ...     ...        ...
45   24.0   764.0   0.031414
83   22.0   943.0   0.023330
16    9.0   664.0   0.013554
55    3.0   230.0   0.013043
81    6.0   505.0   0.011881

[100 rows x 3 columns]


In [272]:
problem = problem_df.to_numpy().T
problem = list(problem)
problem.append(capacity)

In [273]:
value = problem[0]
weight = problem[1]
ratio = problem[2]

In [274]:
def pegging(problem):
    relax_cnt = 0
    
    greedy_v, greedy_ans = greedy(problem)
    relax_v, relax_ans = relaxation(problem, [])
    ans = []
    for i, tf in enumerate(tqdm(relax_ans)):
        
        if tf == 1:
            relax_cnt += 1
            temp_v, temp_ans = relaxation(problem, ans + [[i, 0]])
            if temp_v < greedy_v:
                ans.append([i, 1])
        
        elif tf == 0:
            relax_cnt += 1
            temp_v, temp_ans = relaxation(problem, ans + [[i, 1]])
            if temp_v < greedy_v:
                ans.append([i, 0])
    print(relax_cnt)
    return prued_relax(problem, ans)

In [275]:
#%time enumeration()

In [276]:
%time pegging(problem)


99
163
CPU times: user 212 ms, sys: 10.9 ms, total: 222 ms
Wall time: 222 ms


[32554.0,
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [277]:
%time prued_relax(problem, [])

1810
CPU times: user 1.03 s, sys: 10.1 ms, total: 1.04 s
Wall time: 1.05 s


[32554.0,
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

釘打ちテストはナップサック問題のrationの種類が多いと単純な分枝限定法より高速になる。
しかし、単純な問題であると単純な分枝限定法の方が高速になる